In [1]:
import json
import numpy as np
import os
import faiss
from pdfminer.high_level import extract_text
from sentence_transformers import SentenceTransformer
# from huggingface_hub import hf_hub_download



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [1]:
!pip3 install fitz
!pip3 install PyMuPDF


In [15]:
# Load a pre-trained embedding model
# This line initializes the SentenceTransformer model 'all-MiniLM-L6-v2', a compact, 
# efficient model for creating sentence embeddings that capture semantic meaning.
model = SentenceTransformer('all-MiniLM-L6-v2')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
import fitz  # PyMuPDF
import json
import os
from sentence_transformers import SentenceTransformer

# Load a pre-trained embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

def generate_embeddings(text):
    """Generate embeddings for the given text."""
    return model.encode(text, convert_to_tensor=True)

def extract_pdf_text(pdf_path):
    """Extract text from a PDF file using PyMuPDF."""
    try:
        # Open the PDF file
        doc = fitz.open(pdf_path)
        text = ""
        # Iterate through the pages and extract text
        for page in doc:
            text += page.get_text()
        doc.close()
        return text
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        return ""

def load_metadata(jsonl_file):
    """Load metadata from a JSONL file."""
    try:
        with open(jsonl_file, 'r') as f:
            return [json.loads(line) for line in f]
    except Exception as e:
        print(f"Error loading metadata from {jsonl_file}: {e}")
        return []

def save_extracted_text(text, output_path):
    """Save extracted text to a text file."""
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(text)
    print(f"Extracted text saved to: {output_path}")

# Example usage
def main(pdf_files, metadata_file):
    metadata = load_metadata(metadata_file)  # Load metadata

    # Extract text from each PDF file and generate embeddings
    for pdf_file in pdf_files:
        text = extract_pdf_text(pdf_file)  # Extract text from PDF
        if text:  # Ensure text extraction was successful
            # Generate embeddings (optional)
            embeddings = generate_embeddings(text)  # Generate embeddings
            # Save the extracted text to a new .txt file
            output_file = os.path.splitext(pdf_file)[0] + '_extracted.txt'  # Create output file name
            save_extracted_text(text, output_file)  # Save extracted text

# Define your PDF files and metadata file
pdf_files = ['/Users/innovapathinc/Desktop/Response_eval/data/3M_2015_10K.pdf', '/Users/innovapathinc/Desktop/Response_eval/data/3M_2016_10K.pdf']  # Update paths
metadata_file = '/Users/innovapathinc/Desktop/Response_eval/data/financebench_document_information.jsonl'  # Update with your JSONL metadata file

if __name__ == "__main__":
    main(pdf_files, metadata_file)


Extracted text saved to: /Users/innovapathinc/Desktop/Response_eval/data/3M_2015_10K_extracted.txt
Extracted text saved to: /Users/innovapathinc/Desktop/Response_eval/data/3M_2016_10K_extracted.txt


In [4]:
# Define your paths
jsonl_file = "/Users/innovapathinc/Desktop/Response_eval/data/financebench_document_information.jsonl"
pdf_files = [
    "/Users/innovapathinc/Desktop/Response_eval/data/3M_2015_10K.pdf",
    "/Users/innovapathinc/Desktop/Response_eval/data/3M_2016_10K.pdf"
]

# Load metadata from JSONL file
metadata = load_metadata(jsonl_file)
print("Loaded metadata:", metadata)


Loaded metadata: [{'doc_name': '3M_2015_10K', 'company': '3M', 'gics_sector': 'Industrials', 'doc_type': '10k', 'doc_period': 2015, 'doc_link': 'https://investors.3m.com/financials/sec-filings/content/0001558370-16-003162/0001558370-16-003162.pdf'}, {'doc_name': '3M_2016_10K', 'company': '3M', 'gics_sector': 'Industrials', 'doc_type': '10k', 'doc_period': 2016, 'doc_link': 'https://investors.3m.com/financials/sec-filings/content/0001558370-17-000479/0001558370-17-000479.pdf'}, {'doc_name': '3M_2017_10K', 'company': '3M', 'gics_sector': 'Industrials', 'doc_type': '10k', 'doc_period': 2017, 'doc_link': 'https://investors.3m.com/financials/sec-filings/content/0001558370-18-000535/0001558370-18-000535.pdf'}, {'doc_name': '3M_2018_10K', 'company': '3M', 'gics_sector': 'Industrials', 'doc_type': '10k', 'doc_period': 2018, 'doc_link': 'https://investors.3m.com/financials/sec-filings/content/0001558370-19-000470/0001558370-19-000470.pdf'}, {'doc_name': '3M_2022_10K', 'company': '3M', 'gics_sec

In [9]:
# Initialize FAISS index
index = faiss.IndexFlatL2(384)  # Dimension should match your embedding size

# Extract text from all PDF files and store embeddings
for pdf_file in pdf_files:
    pdf_name = os.path.basename(pdf_file)
    pdf_text = extract_pdf_text(pdf_file)
    embedding = generate_embeddings(pdf_text).cpu().numpy()  # Convert to numpy array
    index.add(np.array([embedding]))  # Add to FAISS index
    print(f"Stored embedding for {pdf_name} in the index.")


NameError: name 'faiss' is not defined